## Ejercicio/Tarea

### Victor Quintero Mármol González    175897

Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo.

* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/ 

* ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

**Carga de librerías:**

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

import pandas as pd

from dask import dataframe
from dask import delayed
from dask_ml.linear_model import LinearRegression
from dask_ml.model_selection import GridSearchCV as GridSearchCV_Dask
from dask.distributed import Client
client = Client("scheduler:8786")


**Lectura y limpieza de datos:**

In [28]:
#Cargamos los datos 
trips_df = pd.read_csv("/data/trips.csv")
# Truco para convertir a datetime...
trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')
#Nos quedamos únicamente con observaciones que tengan "fare_amount"
trips_df = trips_df[trips_df.fare_amount > 0]
#Creamos variable de proporción de propina
trips_df["prop_prop"] = round(trips_df["tip_amount"]/trips_df["fare_amount"],3)
#Creamos variable de duración del viaje en segundos
trips_df["duracion_trip"] = trips_df["tpep_dropoff_datetime"] - trips_df["tpep_pickup_datetime"]
trips_df["duracion_trip"] = trips_df["duracion_trip"].map(lambda x:x.total_seconds())
#Revisamos los datos
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,prop_prop,duracion_trip
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.209,1170.0
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.000,587.0
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.133,627.0
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,0.118,551.0
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,0.221,568.0


In [29]:
#Vemos que hace falta cambiar la variable "car_type" a binario
label_encoder = LabelEncoder()
trips_df["car_type"] = label_encoder.fit_transform(trips_df['car_type'])
#Nos quedamos únicamente con las variables que nos interesan
trips_df = trips_df.drop(["taxi_id", "tip_amount", "tpep_dropoff_datetime", "tpep_pickup_datetime"], axis= 1)
trips_df.head()

,car_type,fare_amount,passenger_count,trip_distance,prop_prop,duracion_trip
0,0,22.0,1,6.90,0.209,1170.0
1,0,9.0,1,1.81,0.000,587.0
2,0,7.5,1,0.96,0.133,627.0
3,0,8.5,1,1.90,0.118,551.0
4,0,7.5,1,1.00,0.221,568.0


**Modelos:**

In [30]:
#Separamos en entrenamiento y prueba
X = trips_df.drop("prop_prop", axis = 1)
y = trips_df["prop_prop"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

In [31]:
#Creamos pipeline. Primero estandarizamos y luego aplicamos regresión Ridge
pipe = Pipeline([('ss', StandardScaler()),('rd', Ridge())])
pipe.fit(X_train, y_train)
#Parámetros del grid
parametros = {'rd__alpha': [2, 1,0.5,0.1],'rd__tol': [0.001,0.0001]}
#Creamos GridSearch
grid = GridSearchCV(estimator=pipe, param_grid=parametros,scoring='neg_mean_squared_error', cv=10)

**Secuencial:**

In [32]:
%%time
#Corremos de manera secuencial
grid.fit(X_train, y_train)

CPU times: user 873 ms, sys: 755 ms, total: 1.63 s
Wall time: 923 ms


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('rd', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'rd__tol': [0.001, 0.0001], 'rd__alpha': [2, 1, 0.5, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [33]:
#Vemos los parámetros del mejor estimador
print(grid.best_estimator_)

Pipeline(memory=None,
     steps=[('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('rd', Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])


**Paralelo:**

In [34]:
%%time
#Corremos de manera paralela
delayed(grid.fit(X_train, y_train))

CPU times: user 901 ms, sys: 656 ms, total: 1.56 s
Wall time: 881 ms


Delayed('GridSearchCV-de3a39a2-f034-49ea-a064-b03c57bb619a')

**Dask-ML:**

In [38]:
#Separamos en entrenamiento y prueba
X = trips_df.drop("prop_prop", axis = 1).values
y = trips_df["prop_prop"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

In [39]:
#Creamos pipeline
pipe_dask = Pipeline([('ss', StandardScaler()),('rd', LinearRegression())])
#Parámetros del grid
parametros_dask = {'rd__C': [2, 1,0.5,0.1],'rd__tol': [0.001,0.0001]}
#Creamos GridSearch
grid_dask = GridSearchCV_Dask(pipe_dask, parametros_dask,scoring='neg_mean_squared_error',cv=10)

In [40]:
%%time
#Corremos dask ml
grid_dask.fit(X_train,y_train)

CPU times: user 417 ms, sys: 55.5 ms, total: 472 ms
Wall time: 1min 9s


GridSearchCV(cache_cv=True, cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('rd', LinearRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multiclass='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False))]),
       iid=True, n_jobs=-1,
       param_grid={'rd__tol': [0.001, 0.0001], 'rd__C': [2, 1, 0.5, 0.1]},
       refit=True, return_train_score='warn', scheduler=None,
       scoring='neg_mean_squared_error')

In [41]:
#Vemos los parámetros del mejor estimador
print(grid_dask.best_estimator_)

Pipeline(memory=None,
     steps=[('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('rd', LinearRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multiclass='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.001, verbose=0, warm_start=False))])


**Comentarios finales:**
* Se pudo observar que entre el secuencial y paralelo, el paralelo es ligeramente más rápido en la ejecución.
* Al usar la librería Dask-ML se puede observar que aunque la ejecución del código en CPU time es la menor, el tiempo de espera en que el que se distribuye la tarea es muy grande, como se puede observar en el Wall time. Esto puede deberse a que la base no es lo suficientemente grande como para sacar provecho de este computo distribuido.